In [1]:
import os
import os.path as osp
import sys
import fire
import json
from typing import List, Union

import pandas as pd

import torch
from torch.nn import functional as F

import transformers
from transformers import TrainerCallback, TrainingArguments, TrainerState, TrainerControl
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR
from transformers import LlamaForCausalLM, LlamaTokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer

from datasets import Dataset

from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_int8_training,
    set_peft_model_state_dict
)
from peft import PeftModel

In [2]:
device = 'auto'
base_LLM_model = 'beomi/OPEN-SOLAR-KO-10.7B'

In [3]:
model = AutoModelForCausalLM.from_pretrained(
    base_LLM_model,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map=device)

tokenizer = AutoTokenizer.from_pretrained(base_LLM_model)

config.json:   0%|          | 0.00/658 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.8k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/966 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [4]:
bos = tokenizer.bos_token_id
eos = tokenizer.eos_token_id
pad = tokenizer.pad_token_id
tokenizer.padding_side = "right"

print("BOS token:", bos)
print("EOS token:", eos)
print("PAD toekn:", pad)

BOS token: 1
EOS token: 2
PAD toekn: None


In [5]:
if pad == None or pad == eos:
    tokenizer.pad_token_id = 0
print("length of tokenizer:", len(tokenizer))

length of tokenizer: 32000


In [6]:
print(model)
print(type(model))

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-47): 48 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear8bitLt(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear8bitLt(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear8bitLt(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
 

In [7]:
batch_size = 16
num_epochs = 1
micro_batch = 2
gradient_accumulation_steps = batch_size // micro_batch

cutoff_len = 4096
lr_scheduler = 'cosine'
warmup_ratio = 0.06
learning_rate = 1e-4
optimizer = 'adamw_torch'
weight_decay = 0.01
max_grad_norm = 1.0

lora_r = 16
lora_alpha = 16
lora_dropout = 0.05
lora_target_modules = ["gate_proj", "down_proj", "up_proj"]

train_on_inputs = False
add_eos_token = False

resume_from_checkpoint = False
output_dir = './custom_LLM'

In [8]:
train_df = pd.read_csv('data/train.csv')
train_df.head()

,id,질문_1,질문_2,category,답변_1,답변_2,답변_3,답변_4,답변_5
0,TRAIN_000,면진장치가 뭐야?,면진장치에 사용되는 주요 기술은 무엇인가요?,건축구조,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...,"면진장치란 건물의 지반에서 발생하는 진동 에너지를 흡수하여 건물을 보호하고, 진동을...",면진장치란 지반으로부터 발생하는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여...,면진장치는 건물의 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 ...,면진장치는 건물에 오는 지반 진동의 영향을 최대한으로 흡수하여 건물에 전달되는 진동...
1,TRAIN_001,내진설계의 종류 좀 알려줘,내진설계에는 어떤 종류가 있는지 자세히 알려주실 수 있나요?,건축구조,"내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다.","내진설계에는 내진구조, 제진구조, 면진구조가 있습니다. 내진구조는 건물 구조물이 지...","내진설계에는 주로 내진구조, 제진구조, 면진구조의 세 가지 종류가 있습니다. 이들은...","내진설계에는 주로 내진구조, 제진구조, 면진구조가 사용됩니다. 내진구조는 건물 구조...","내진 설계에는 다양한 종류가 있지만, 대표적으로 내진구조, 제진구조, 면진구조가 있..."
2,TRAIN_002,철골구조의 장점이 뭐야?,철골구조의 장점을 알려줘?,건축구조,철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건축이 가...,철골구조의 장점은 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건...,철골구조의 장점은 건물의 외벽에 하중이 적게 걸리기 때문에 고층 건물의 건축이 용이...,"철골구조의 장점은 건물의 외벽이 하중이 걸리지 않아 공간 활용이 용이하고, 고층 건...",철골구조의 장점은 건물의 외벽에 하중이 크게 걸리지 않아 고층 건물을 건축할 수 있...
3,TRAIN_003,철골철근 콘크리트 구조가 뭐야?,철골철근 콘크리트 구조의 장점과 단점에는 무엇이 있을까요?,건축구조,"철근철골콘크리트는 철골과 철근, 그리고 콘크리트를 함께 사용하는 건축 구조입니다. ...","철골철근콘크리트 구조는 건축물을 지탱하는 주요 구조물인 철골과 철근, 그리고 콘크리...",철골철근 콘크리트 구조는 건축물을 지탱하기 위한 구조물에서 일반적으로 사용되는 방식...,"철골철근콘크리트 구조는 철골과 철근, 그리고 콘크리트를 함께 사용하여 만들어지는 건...","철골철근 콘크리트 구조는 강철 골조와 강철 철근, 그리고 콘크리트를 함께 사용하여 ..."
4,TRAIN_004,철골구조는 어떤 방식이 있어?,철골구조의 다양한 방식이 무엇인가요?,건축구조,철골구조는 일반철골구조와 경량철골구조가 있습니다.,철골구조는 일반철골구조와 경량철골구조가 있습니다. 일반철골구조는 주로 대형 건물이나...,철골구조는 주로 일반철골구조와 경량철골구조로 나뉘어집니다. 이들은 건축 시스템에 따...,철골구조는 주로 일반철골구조와 경량철골구조로 구분됩니다. 이외에도 최근에는 고층 건...,철골구조는 일반철골구조와 경량철골구조 두 가지 방식이 주로 사용됩니다. 일반철골구조...


In [9]:
def train_generator():
  for i, row in train_df.iterrows():
    questions = row[['질문_1', '질문_2']].values
    answers = row[['답변_1', '답변_2', '답변_3', '답변_4', '답변_5']].values
    for question in questions:
      for answer in answers:
        yield dict(question=question, answer=answer)

In [10]:
train_dataset = Dataset.from_generator(train_generator)
train_dataset

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['question', 'answer'],
    num_rows: 6440
})

In [11]:
from string import Template

prompt_template = Template(
    """### System:
${system}

### User:
${user}

### Assistant:
""")

In [12]:
def preprocess(example):
  prompt = prompt_template.substitute(
      system="당신은 건물 도배 전문가로써 User의 질문에 알맞은 답변을 제시하세요.",
      user=example['question']
  )
  prompt_with_answer = prompt + example['answer']

  full_tokenized = tokenizer(prompt_with_answer)

  # add eos token
  full_tokenized['input_ids'].append(eos)
  full_tokenized['attention_mask'].append(1)

  nums_prompt_tokens = len(tokenizer.encode(prompt))

  full_tokenized['labels'] = [-100] * nums_prompt_tokens + full_tokenized['input_ids'][nums_prompt_tokens:]

  return full_tokenized

In [13]:
val_dataset = None
train_dataset = train_dataset.shuffle()
train_dataset = train_dataset.map(preprocess)

Map:   0%|          | 0/6440 [00:00<?, ? examples/s]

In [14]:
config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    target_modules=lora_target_modules,
    lora_dropout=lora_dropout,
    bias="none",
    task_type="CAUSAL_LM"
)

In [15]:
model = prepare_model_for_int8_training(model)
model = get_peft_model(model, config)

/opt/conda/lib/python3.10/site-packages/peft/utils/other.py:143: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


In [16]:
if resume_from_checkpoint:
    checkpoint_name = os.path.join(
        resume_from_checkpoint, "pytorch_model.bin"
    )

    if not os.path.exists(checkpoint_name):
        checkpoint_name = os.path.join(
            resume_from_checkpoint, "adapter_model.bin"
        )
        resume_from_checkpoint = (
            True
        )
    
    if os.path.exists(checkpoint_name):
        print(f"Restarting from {checkpoint_name}")
        adapters_weights = torch.load(checkpoint_name)
        set_peft_model_state_dict(model, adapters_weights)
    else:
        print(f"Checkpoint {checkpoint_name} not found")

In [17]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=micro_batch,
        gradient_accumulation_steps=gradient_accumulation_steps,
        warmup_ratio=warmup_ratio,
        num_train_epochs=num_epochs,
        learning_rate=learning_rate,
        fp16=True,
        logging_steps=1,
        optim="adamw_torch",
        evaluation_strategy="no",
        save_strategy="steps",
        max_grad_norm=max_grad_norm,
        save_steps=30,
        lr_scheduler_type=lr_scheduler,
        output_dir=output_dir,
        save_total_limit=2,
        load_best_model_at_end=False,
        ddp_find_unused_parameters=False,
        group_by_length=False,
    ),
    data_collator=transformers.DataCollatorForSeq2Seq(
        tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True,
    ),
)

model.config.use_cache = False
model.print_trainable_parameters()

if torch.__version__ >= "2" and sys.platform != "win32":
    model = torch.compile(model)

trainable params: 42,467,328 || all params: 10,773,991,424 || trainable%: 0.39416522928912257


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [18]:
torch.cuda.empty_cache()
trainer.train(resume_from_checkpoint=resume_from_checkpoint)

/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
1,1.529700
2,1.373300
3,1.576500
4,1.676300
5,1.595700
6,1.572200
7,1.413700
8,1.416500
9,1.208300
10,1.443800


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain 

TrainOutput(global_step=805, training_loss=0.5788128519280357, metrics={'train_runtime': 11360.2912, 'train_samples_per_second': 0.567, 'train_steps_per_second': 0.071, 'total_flos': 1.1563676697880166e+17, 'train_loss': 0.5788128519280357, 'epoch': 1.0})

In [19]:
model.save_pretrained(output_dir)
model_path = os.path.join(output_dir, "pytorch_model.bin")
torch.save({}, model_path)
tokenizer.save_pretrained(output_dir)

('./custom_LLM/tokenizer_config.json',
 './custom_LLM/special_tokens_map.json',
 './custom_LLM/tokenizer.model',
 './custom_LLM/added_tokens.json',
 './custom_LLM/tokenizer.json')

In [20]:
torch.cuda.empty_cache()

base_model = AutoModelForCausalLM.from_pretrained(
    base_LLM_model,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device
)

model = PeftModel.from_pretrained(base_model, output_dir, device)
model = model.merge_and_unload()

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [21]:
final_save_folder = './custom_LLM_final'

model.save_pretrained(final_save_folder)
tokenizer.save_pretrained(final_save_folder)

('./custom_LLM_final/tokenizer_config.json',
 './custom_LLM_final/special_tokens_map.json',
 './custom_LLM_final/tokenizer.model',
 './custom_LLM_final/added_tokens.json',
 './custom_LLM_final/tokenizer.json')

In [25]:
torch.cuda.empty_cache()
model = AutoModelForCausalLM.from_pretrained(final_save_folder)
model.push_to_hub('open-solar-hansol-lr1e-4', token=True)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

SafetensorError: Error while serializing: IoError(Os { code: 28, kind: StorageFull, message: "No space left on device" })

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(final_save_folder)
tokenizer.push_to_hub('open-solar-hansol-lr1e-4', token=True)